In [27]:
import requests
import pandas as pd
import json
import xml.etree.ElementTree as ET
from io import StringIO
import os

In [2]:
hotness = 'https://boardgamegeek.com/xmlapi2/hot?type=boardgame'

In [3]:
response = requests.get('https://boardgamegeek.com/api/reviews/169786')

In [4]:
response

<Response [404]>

In [5]:
url = "https://boardgamegeek.com/api/reviews/169786"
response = requests.get(url)
response


<Response [404]>

In [6]:
response = requests.get('https://boardgamegeek.com/xmlapi2/hot?type=boardgame')
response

<Response [200]>

In [7]:
response.content

b'<?xml version="1.0" encoding="utf-8"?><items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse">\n\t\t\t<item id="441696" rank="1">\n\t\t\t<thumbnail value="https://cf.geekdo-images.com/D4tnUD7cJ2e6cUX9ssceZQ__thumb/img/hH25u2q2iKlHnORpEAl8mDxNJAU=/fit-in/200x150/filters:strip_icc()/pic8783245.jpg"/>\n\t\t\t<name value="Sanctuary"/>\t\t\t\n\t\t\t<yearpublished value="2025" />\t\t</item>\n\t\t\t<item id="244521" rank="2">\n\t\t\t<thumbnail value="https://cf.geekdo-images.com/B1bLRWzTASZ-xx9NoAE79A__thumb/img/deG3pr-ORZ3JrGvQA-oQNrwh0l0=/fit-in/200x150/filters:strip_icc()/pic8780293.png"/>\n\t\t\t<name value="Quacks"/>\t\t\t\n\t\t\t<yearpublished value="2018" />\t\t</item>\n\t\t\t<item id="441706" rank="3">\n\t\t\t<thumbnail value="https://cf.geekdo-images.com/3zMXvAfZbnWR2UIjbNMIYA__thumb/img/YpTgFEY6Wl71VLazjE_3aElfCPM=/fit-in/200x150/filters:strip_icc()/pic8780291.png"/>\n\t\t\t<name value="Quacks: All-In Edition"/>\t\t\t\n\t\t\t<yearpublished value="2025" />\t\t</item>\n\t\t\

In [8]:
ET.fromstring(response.content)

<Element 'items' at 0x7b041dfc95d0>

In [9]:
xml_data = StringIO(response.text)

df = pd.read_xml(xml_data)

In [10]:
df

,id,rank,thumbnail,name,yearpublished
0,441696,1,NaN,NaN,NaN
1,244521,2,NaN,NaN,NaN
2,441706,3,NaN,NaN,NaN
3,418059,4,NaN,NaN,NaN
4,424785,5,NaN,NaN,NaN
5,372649,6,NaN,NaN,NaN
6,349750,7,NaN,NaN,NaN
7,342942,8,NaN,NaN,NaN
8,391137,9,NaN,NaN,NaN
9,434654,10,NaN,NaN,NaN


In [28]:
file = '../data/games_comments.csv'

if os.path.exists(file):
    os.remove(file)  # Delete the file
    print(f"{file} has been removed.")
else:
    print(f"{file} does not exist.")

for id in df['id'] :
    url = f'https://boardgamegeek.com/xmlapi/boardgame/{id}?&comments=1&stats=1'

    response = requests.get(url)

    print(response)
    xml_data = StringIO(response.text)

    xml_data = StringIO(response.text)

    # Define namespaces if needed (adjust based on XML structure, remove if no namespaces)
    namespaces = {'ns': 'http://www.boardgamegeek.com/xmlapi'}

    # Extract comments using pandas 
    comments = pd.read_xml(xml_data, xpath=".//comment", namespaces=namespaces)

    # Reset the StringIO buffer
    xml_data = StringIO(response.text)

    # Extract ratings statistics
    ratings = pd.read_xml(xml_data, xpath=".//ratings", namespaces=namespaces)
    # comments = pd.read_xml(xml_data, xpath='.//comment')

    # response = requests.get(f'https://boardgamegeek.com/xmlapi/boardgame/{id}?comments=1')

    # print(response)
    # xml_data = StringIO(response.text)
    # ratings = pd.read_xml(xml_data, xpath=".//boardgame/ratings")
            
    xml_data = StringIO(response.text)
    tmp = pd.read_xml(xml_data, xpath=".//boardgame")
    
    
    # xml_data = StringIO(response.text) 
    min_players = tmp['minplayers'] #pd.read_xml(xml_data, xpath=".//minplayers")
    max_players = tmp['maxplayers'] #pd.read_xml(xml_data, xpath=".//maxplayers")

    # xml_data = StringIO(response.text)
    minplaytime = tmp['minplaytime'] #pd.read_xml(xml_data, xpath=".//minplaytime")
    maxplaytime = tmp['maxplaytime'] #pd.read_xml(xml_data, xpath=".//maxplaytime")

    # xml_data = StringIO(response.text)
    age = tmp['age'] #pd.read_xml(xml_data, xpath=".//age")

    description = tmp['description']
    
    comments['max_players'] = min_players.values[0]
    comments['max_players'] = max_players.values[0]

    comments['minplaytime'] = minplaytime.values[0]
    comments['maxplaytime'] = maxplaytime.values[0]

    comments['age'] = age[0]

    comments['ratings_avg'] = ratings['average'].values[0]
    
    comments['count_wanting'] = ratings['wanting'].values[0]
    comments['count_wishing'] = ratings['wishing'].values[0]

    comments.to_csv(file, mode='a', header=True, index=False)
    

../data/games_comments.csv does not exist.
<Response [200]>
1 5
40 100
12
   usersrated  average  bayesaverage                              ranks  \
0           2      3.5             0  \n\t\t\t   \t\t\t\t   \t\t\t\t      

   stddev  median  owned  trading  wanting  wishing  numcomments  numweights  \
0     2.5       0      1        0       44      460            7           1   

   averageweight  
0              5  
         username  rating                                            comment
0  causticforever     NaN  Played prototype- will be an enjoyable way to ...
1       Corwin007     NaN                         UPCOMING\n\nArk Nova lite?
2     IronTarkles     NaN                    New game from ark nova designer
3          MarkyX     NaN  I'm very interested in this one. I like the co...
4      mikamikomi     1.0       3 artist yet still use stock photos? oh yeah
5    pauloantunes     NaN                                   Simple Ark Nova.
6           Surya     NaN  Lighter ve

In [26]:
comments

,username,rating,comment,max_players,minplaytime,maxplaytime,age,ratings_avg,count_wanting,count_wishing
0,causticforever,NaN,Played prototype- will be an enjoyable way to ...,5,40,100,12,3.5,44,460
1,Corwin007,NaN,UPCOMING\n\nArk Nova lite?,5,40,100,12,3.5,44,460
2,IronTarkles,NaN,New game from ark nova designer,5,40,100,12,3.5,44,460
3,MarkyX,NaN,I'm very interested in this one. I like the co...,5,40,100,12,3.5,44,460
4,mikamikomi,1.0,3 artist yet still use stock photos? oh yeah,5,40,100,12,3.5,44,460
5,pauloantunes,NaN,Simple Ark Nova.,5,40,100,12,3.5,44,460
6,Surya,NaN,Lighter version of Ark Nova. Does not sound li...,5,40,100,12,3.5,44,460
7,Wrathberry,9.0,rating after 1 play of a very late prototype.\...,5,40,100,12,3.5,44,460
